In [1]:
% load_ext autoreload
% autoreload 2

UsageError: Line magic function `%` not found.


In [3]:
import sys

sys.path.append("../")

In [4]:
from argparse import ArgumentParser
from pytorch_lightning import Trainer
from entlink.model.efficient_el import EfficientEL
from entlink.utils import get_markdown

from IPython.display import Markdown

In [6]:
parser = ArgumentParser()

parser = Trainer.add_argparse_args(parser)

args, _ = parser.parse_known_args()
args.gpus = 0
args.precision = 16

trainer = Trainer.from_argparse_args(args)

/Users/yasas/Documents/Projects/ysenarath/efficient-autoregressive-EL/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:658: UserWarning: You passed `Trainer(accelerator='cpu', precision=16)` but native AMP is not supported on CPU. Using `precision='bf16'` instead.
  rank_zero_warn(
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [18]:
kwargs = {
    'test_data_path': '../data/aida_test_dataset.jsonl',
    'mentions_filename': '../data/mentions.json',
    'entities_filename': '../data/entities.json',
}

model = EfficientEL.load_from_checkpoint('../models/model.ckpt', **kwargs).eval()

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForMaskedLM: ['longformer.encoder.layer.9.attention.self.value_global.weight', 'longformer.encoder.layer.8.attention.self.query_global.bias', 'longformer.encoder.layer.10.attention.self.key.bias', 'longformer.encoder.layer.10.attention.self.value.weight', 'longformer.encoder.layer.9.attention.self.key.weight', 'longformer.encoder.layer.11.attention.output.dense.bias', 'longformer.encoder.layer.11.attention.self.value.bias', 'longformer.encoder.layer.8.output.dense.bias', 'longformer.encoder.layer.8.output.dense.weight', 'longformer.encoder.layer.11.attention.self.value_global.bias', 'longformer.encoder.layer.8.intermediate.dense.bias', 'longformer.encoder.layer.8.attention.output.dense.bias', 'longformer.encoder.layer.11.attention.self.query.weight', 'longformer.encoder.layer.11.attention.self.key_global.bias', 'longformer.encoder.layer.9.attention.self.query_global.weight', '

In [19]:
model.hparams.threshold = -3.2
model.hparams.test_with_beam_search = False
model.hparams.test_with_beam_search_no_candidates = False
# trainer.test(model, test_dataloaders=model.test_dataloader(), ckpt_path=None)

In [20]:
model.generate_global_trie()

Loading ..:   0%|          | 0/470105 [00:00<?, ?it/s]

In [39]:
s = "American Airlines is a fine airline."

model.sample([s])

[[(0,
   17,
   [('United States', -2.541071891784668),
    ('United Airlines', -4.048898220062256),
    ('United Kingdom', -4.068247318267822),
    ('American Airlines', -6.304810047149658),
    ('United States Department of Agriculture', -12.176301002502441)])]]

In [41]:
Markdown(get_markdown([s], [[(s[0], s[1], s[2][0][0]) for s in spans] for spans in model.sample([s])])[0])

[American Airlines](https://en.wikipedia.org/wiki/United_States) is a fine airline.